## Map data
heatmap by counties, weights as pollution data

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
import requests
import json
import copy
from typing import List
from urllib.request import urlretrieve
from urllib.parse import quote
import csv
import gmaps
import os
from config import g_key
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Import API key
from config import key
from config import email_key

In [2]:
# Import from .CSV
california_df = pd.read_csv("./csv_data/california_losangeles.csv") 

In [3]:
california_df.head()

,state_code,state,county_code,county,site_number,parameter,sample_measurement,units_of_measure,latitude,longitude,date_local,time_local,date_gmt,time_gmt,method_type,method_code,method,date_of_last_change,cbsa_code
0,6,California,37,Los Angeles,2,PM2.5 - Local Conditions,3.9,Micrograms/cubic meter (LC),34.1365,-117.92391,2019-01-06,00:00,2019-01-06,08:00,FRM,143,R & P Model 2000 PM-2.5 Air Sampler w/VSCC - G...,2019-06-06,31080
1,6,California,37,Los Angeles,2,PM2.5 - Local Conditions,3.8,Micrograms/cubic meter (LC),34.1365,-117.92391,2019-01-12,00:00,2019-01-12,08:00,FRM,143,R & P Model 2000 PM-2.5 Air Sampler w/VSCC - G...,2019-06-06,31080
2,6,California,37,Los Angeles,2,PM2.5 - Local Conditions,8.1,Micrograms/cubic meter (LC),34.1365,-117.92391,2019-01-18,00:00,2019-01-18,08:00,FRM,143,R & P Model 2000 PM-2.5 Air Sampler w/VSCC - G...,2019-06-06,31080
3,6,California,37,Los Angeles,2,PM2.5 - Local Conditions,3.0,Micrograms/cubic meter (LC),34.1365,-117.92391,2019-01-24,00:00,2019-01-24,08:00,FRM,143,R & P Model 2000 PM-2.5 Air Sampler w/VSCC - G...,2019-06-06,31080
4,6,California,37,Los Angeles,2,PM2.5 - Local Conditions,21.2,Micrograms/cubic meter (LC),34.1365,-117.92391,2019-01-30,00:00,2019-01-30,08:00,FRM,143,R & P Model 2000 PM-2.5 Air Sampler w/VSCC - G...,2019-06-06,31080


In [4]:
# Make copy before filtering
california_filtered_df = california_df.copy()

In [5]:
# Select certain columns to use for mapping
california_filtered_df=california_filtered_df[['county','site_number','sample_measurement','latitude','longitude','time_local','date_gmt','time_gmt','date_of_last_change']]
california_filtered_df.head(20)

,county,site_number,sample_measurement,latitude,longitude,time_local,date_gmt,time_gmt,date_of_last_change
0,Los Angeles,2,3.9,34.1365,-117.92391,00:00,2019-01-06,08:00,2019-06-06
1,Los Angeles,2,3.8,34.1365,-117.92391,00:00,2019-01-12,08:00,2019-06-06
2,Los Angeles,2,8.1,34.1365,-117.92391,00:00,2019-01-18,08:00,2019-06-06
3,Los Angeles,2,3.0,34.1365,-117.92391,00:00,2019-01-24,08:00,2019-06-06
4,Los Angeles,2,21.2,34.1365,-117.92391,00:00,2019-01-30,08:00,2019-06-06
5,Los Angeles,2,2.7,34.1365,-117.92391,00:00,2019-02-05,08:00,2019-06-06
6,Los Angeles,2,2.3,34.1365,-117.92391,00:00,2019-02-11,08:00,2019-06-06
7,Los Angeles,2,2.8,34.1365,-117.92391,00:00,2019-02-17,08:00,2019-06-06
8,Los Angeles,2,3.5,34.1365,-117.92391,00:00,2019-02-23,08:00,2019-06-06
9,Los Angeles,2,11.9,34.1365,-117.92391,00:00,2019-03-01,08:00,2019-06-06


In [6]:
california_filtered_df=california_filtered_df.dropna()

In [7]:
california_filtered_df = california_filtered_df[(california_filtered_df[['sample_measurement']] > 0).all(axis=1)]
california_filtered_df.head()

,county,site_number,sample_measurement,latitude,longitude,time_local,date_gmt,time_gmt,date_of_last_change
0,Los Angeles,2,3.9,34.1365,-117.92391,00:00,2019-01-06,08:00,2019-06-06
1,Los Angeles,2,3.8,34.1365,-117.92391,00:00,2019-01-12,08:00,2019-06-06
2,Los Angeles,2,8.1,34.1365,-117.92391,00:00,2019-01-18,08:00,2019-06-06
3,Los Angeles,2,3.0,34.1365,-117.92391,00:00,2019-01-24,08:00,2019-06-06
4,Los Angeles,2,21.2,34.1365,-117.92391,00:00,2019-01-30,08:00,2019-06-06


In [8]:
california_filtered_df.describe()

,site_number,sample_measurement,latitude,longitude
count,18338.000000,18338.000000,18338.000000,18338.000000
mean,5986.528302,10.225134,34.222417,-118.168705
std,2754.527494,8.474642,0.394892,0.050755
min,2.000000,0.100000,33.792360,-118.532760
25%,4008.000000,5.000000,33.859662,-118.200707
50%,4008.000000,8.400000,33.901389,-118.200707
75%,9033.000000,13.000000,34.669739,-118.130511
max,9033.000000,360.400000,34.669739,-117.923910


In [9]:
# Set Index
california_filtered_df=california_filtered_df.set_index('date_of_last_change')

# Sort Values
ca_dates = california_filtered_df.sort_values(by='date_of_last_change', axis=0).index.unique()

## Begin Heat Map. 

In [10]:
# Single out Lat,Lon values to combine into list
lat_cords = [i for i in california_filtered_df['latitude']]
lon_cords = [i for i in california_filtered_df['longitude']]
# Bring out humit values. Used for weights in gmaps
humid_weights = california_filtered_df['sample_measurement'].tolist()

In [11]:
# Go through two lists to create [(lat,lng)] for coordinate values. 
# Used in gmaps
coordinates = []
for i,j in zip(lat_cords, lon_cords):
    coords = (i,j)
    coordinates.append(coords)

In [12]:
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Add Heatmap layer to map.
heat_layer = gmaps.heatmap_layer(coordinates, weights=humid_weights, 
                                 dissipating=False, max_intensity=.4,
                                 point_radius=.1)

#markers = gmaps.marker_layer(coordinates)
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

Edited to emphasize weights

In [13]:
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Add Heatmap layer to map.
heat_layer = gmaps.heatmap_layer(coordinates, weights=humid_weights, 
                                 dissipating=False, max_intensity=.99,
                                 point_radius=.2)

#markers = gmaps.marker_layer(coordinates)
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…